# Learning 
### At first we gonna read the data from pickle file in put the images data in X and put the lable in y 

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D , BatchNormalization
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
import datetime
from tensorflow import keras

%load_ext tensorboard
!rm -rf ./logs/ 

X = pickle.load(open("./pickle/X_learn.pickle", "rb"))
y = pickle.load(open("./pickle/y_learn.pickle", "rb"))
print(X.shape)

### In this part we gonna normalize the data to have the best result

In [ ]:
X = tf.keras.utils.normalize(X, axis=1)

### Now it is time to making CNN model for our learning 

In [ ]:
model = Sequential()

model.add(Conv2D(256, (3, 3),activation="relu", input_shape=X.shape[1:],strides=(1, 1), padding='valid'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3),activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3),activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation=tf.nn.relu))
model.add(Dropout(0.25))

model.add(Dense(10, activation=tf.nn.softmax))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 256)       7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 256)       590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 512)         1180160   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 512)         2

### This class is used for CallBack of model in order to stop the model learning when the model reach to a good accuricy (70% in here)

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        acc = logs["val_accuracy"]
        if acc>0.7:
          self.model.stop_training = True

### In this part we gonna start epoching and then validating until getting to a good result (in here 70% of accuricy)

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

X_train = pickle.load(open("./pickle/X_train.pickle", "rb"))
y_train = pickle.load(open("./pickle/y_train.pickle", "rb"))

X_train = np.array(X_train).reshape(-1, 50, 50, 3)
X_train = tf.keras.utils.normalize(X_train, axis=1)

y_train = np.array(y_train)

model.fit(x = X, y = y,epochs=50,validation_data=(X_train, y_train),callbacks=[CustomCallback(),tensorboard_callback])


### Evaluation of traning data and showing the summery in "TensorBoard"

In [ ]:
val_loss, val_acc = model.evaluate(X_train, y_train)  # evaluate the out of sample data with model
%tensorboard --logdir logs/fit

### In here a number of example of detected image gonna be shown and a plot of number of Correct and Wrong answer

In [ ]:
animals = ["horse", "dog", "elephant", "butterfly", "rooster", "cat", "cow", "tarantula", "squirrel", "sheep"]

predictions = model.predict(X_train)
X2 = pickle.load(open("./pickle/X_train.pickle", "rb"))

result = []
t = 0
f = 0

for i in range(len(predictions)):
    if np.argmax(predictions[i]) == y_train[i]:
        t = t + 1
    else:
        f = f + 1
    result.append([np.argmax(predictions[i]), y_train[i]])

plt.figure(figsize=(10,10))



for i in range(25):   
  num = i * 100 + 1
# print("my pridication is :   " + animals[np.argmax(predictions[num])])
# print("it is :   " + animals[y2[num]])
  plt.subplot(6,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid('off')
  img = cv2.cvtColor(X2[num], cv2.COLOR_BGR2RGB)
  plt.imshow(img)
  plt.xlabel(animals[np.argmax(predictions[num])])

x = ['Correct', 'Wrong']
popularity = [t, f]
x_pos = [i for i, _ in enumerate(x)]

fig, ax = plt.subplots()
rects1 = ax.bar(x_pos, popularity, color='b')
plt.ylabel("The answers")
plt.title("\n\n" + "Neural Network with for detecting animals\n")
plt.xticks(x_pos, x)
# Turn on the grid
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2., 1.05 * height,
                '%f' % float(height),
                ha='center', va='bottom')


autolabel(rects1)
plt.show()

### Saving Model

In [ ]:
model.save('./animal_detector')